# Mini Project

## Project Description
The data scientists at BigMart have collected **2013** sales data for **1559** products across **10** stores in different cities.<br>
Also, certain attributes of each product and store have been defined.<br>
<mark>The aim of this data science project is to build a predictive model and find out the sales of each product at a particular store</mark>.<br>
Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales.<br>
The data has missing values as some stores do not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.

## <div id="top">Project Stages</div>

1. <a href="#data_exploration">Data Exploration</a>
2. <a href="#data_cleaning">Data Cleaning</a><br>
   - <a href="#missing_values" style="color:gray">Handling Missing Values</a>
   - <a href="#duplicates" style="color:gray">Checking for Duplicates</a>
3. <a href="#data_preprocessing">Data Preprocessing</a>
<hr width="70%">


## <div2 id ="data_exploration"><div>Data Exploration</div>

We will start by importing the modules that we need, and have an overview of our data

In [71]:
# Import Needed Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [72]:
# Reading the dataset
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')
train_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [73]:
# The size of our dataset
train_df.shape

(8523, 12)

Now we want to take a closer look at every cloumn we have

In [74]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [75]:
# Showing the unique values for each column
for col in train_df.columns:
    print(col)
    print(train_df[col].unique(), '\nNo. of uniqe values: ', train_df[col].nunique(), '\n')

Item_Identifier
['FDA15' 'DRC01' 'FDN15' ... 'NCF55' 'NCW30' 'NCW05'] 
No. of uniqe values:  1559 

Item_Weight
[ 9.3    5.92  17.5   19.2    8.93  10.395 13.65     nan 16.2   11.8
 18.5   15.1   17.6   16.35   9.    13.35  18.85  14.6   13.85  13.
  7.645 11.65   5.925 19.25  18.6   18.7   17.85  10.     8.85   9.8
 13.6   21.35  12.15   6.42  19.6   15.85   7.39  10.195  9.895 10.895
  7.905  9.195  8.365  7.97  17.7   19.35   8.645 15.6   18.25   7.855
  7.825  8.39  12.85  19.     5.905  7.76  16.75  15.5    6.055  6.305
 20.85  20.75   8.895 19.7    8.75  13.3    8.31  19.75  17.1   10.5
  6.635 14.15   8.89   9.1    7.5   16.85   7.485 11.6   12.65  20.25
  8.6   12.6    8.88  20.5   13.5    7.235  6.92   8.02  12.8   16.6
 14.    16.    21.25   7.365 18.35   5.465  7.27   6.155 19.5   15.2
 14.5   13.1   12.3   11.1   11.3    5.75  11.35   6.525 10.3    5.78
 11.85  18.75   5.26  16.1    9.5   13.8   14.65   6.67   6.11  17.2
  6.32   4.88   5.425 14.1    7.55  17.25  12.    10.

We will notice that:<br>
- `Item_Fat_Content` column have the values [_**Low Fat**_, _**Regular**_, _**low fat**_, _**LF**_, _**reg**_]<br/>
which represent only <span style="color:green">low fat</span> and <span style="color:green">regular</span> content

- `Outlet_Size` and `Item_Weight` column has Nan values

In [76]:
train_df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


There are a strong variations in the data values, so we will consider **normalizing** the data

In [77]:
# Showing the correlation between each column
train_df.corr()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
Item_Weight,1.000000,-0.014048,0.027141,-0.011588,0.014123
Item_Visibility,-0.014048,1.000000,-0.001315,-0.074834,-0.128625
Item_MRP,0.027141,-0.001315,1.000000,0.005020,0.567574
Outlet_Establishment_Year,-0.011588,-0.074834,0.005020,1.000000,-0.049135
Item_Outlet_Sales,0.014123,-0.128625,0.567574,-0.049135,1.000000


<hr width="70%">

<a href="#top">Back to top ⬆</a>
## <span id="data_cleaning">Data Cleaning</span>

First we will handle the redundant values in the `Item_Fat_Content` column

In [78]:
# Unification of the redundant values
train_df['Item_Fat_Content'].replace(inplace=True, to_replace=['Low Fat', 'LF'], value='low fat')
train_df['Item_Fat_Content'].replace(inplace=True, to_replace=['Regular', 'reg'], value='regular')

In [79]:
train_df['Item_Fat_Content'].unique()

array(['low fat', 'regular'], dtype=object)

Doing the same process for the test_df

In [80]:
# Unification
test_df['Item_Fat_Content'].replace(inplace=True, to_replace=['Low Fat', 'LF'], value='low fat')
test_df['Item_Fat_Content'].replace(inplace=True, to_replace=['Regular', 'reg'], value='regular')
test_df['Item_Fat_Content'].unique()

array(['low fat', 'regular'], dtype=object)

Now we have only 2 values: [<span style="color:green">low fat</span>, <span style="color:green">regular</span>]
<hr width=50%>

<a href="#top">Back to top ⬆</a>
### <span id="missing_values">Dealing with Missing Values</span>

In [81]:
# How many missing values do we have
train_df.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

We have missing values in the columns `Item_Weight` and `Outlet_Size`and the number of missing values are so high,<br>
so let's see if there is any reason or pattern for these missing values so we can estimate them.<br>
Lets start with the `Item_Weight` column, we will take  a look at some items from the **Item_Identifier** column to see any patterns.

Let's take the first 3 items for testing

In [82]:
# taking the first 3 items for testing
t_items = train_df['Item_Identifier'].unique()[0:3]
train_df[train_df['Item_Identifier'] == t_items[0]]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.3,low fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
831,FDA15,9.3,low fat,0.016055,Dairy,250.2092,OUT045,2002,NaN,Tier 2,Supermarket Type1,5976.2208
2599,FDA15,9.3,low fat,0.016019,Dairy,248.5092,OUT035,2004,Small,Tier 2,Supermarket Type1,6474.2392
2643,FDA15,9.3,low fat,0.016088,Dairy,249.6092,OUT018,2009,Medium,Tier 3,Supermarket Type2,5976.2208
4874,FDA15,9.3,low fat,0.026818,Dairy,248.9092,OUT010,1998,NaN,Tier 3,Grocery Store,498.0184
5413,FDA15,9.3,low fat,0.016009,Dairy,250.6092,OUT013,1987,High,Tier 3,Supermarket Type1,6474.2392
6696,FDA15,NaN,low fat,0.015945,Dairy,249.5092,OUT027,1985,Medium,Tier 3,Supermarket Type3,6474.2392
7543,FDA15,9.3,low fat,0.016113,Dairy,248.8092,OUT017,2007,NaN,Tier 2,Supermarket Type1,5976.2208


In [83]:
# The 2nd item
train_df[train_df['Item_Identifier'] == t_items[1]]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1,DRC01,5.92,regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2272,DRC01,5.92,regular,0.019309,Soft Drinks,49.0692,OUT017,2007,NaN,Tier 2,Supermarket Type1,1478.0760
2927,DRC01,NaN,regular,0.019107,Soft Drinks,48.4692,OUT027,1985,Medium,Tier 3,Supermarket Type3,1034.6532
5029,DRC01,5.92,regular,0.019239,Soft Drinks,49.8692,OUT045,2002,NaN,Tier 2,Supermarket Type1,1133.1916
5077,DRC01,5.92,regular,0.019200,Soft Drinks,47.7692,OUT046,1997,Small,Tier 1,Supermarket Type1,492.6920
5218,DRC01,5.92,regular,0.019184,Soft Drinks,50.3692,OUT013,1987,High,Tier 3,Supermarket Type1,591.2304


In [84]:
# The 3rd item
train_df[train_df['Item_Identifier'] == t_items[2]]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
2,FDN15,17.5,low fat,0.016760,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.270
759,FDN15,17.5,low fat,0.028009,Meat,141.718,OUT010,1998,NaN,Tier 3,Grocery Store,419.454
4817,FDN15,17.5,low fat,0.016720,Meat,139.918,OUT013,1987,High,Tier 3,Supermarket Type1,419.454
5074,FDN15,17.5,low fat,0.016802,Meat,138.518,OUT018,2009,Medium,Tier 3,Supermarket Type2,2376.906
6163,FDN15,17.5,low fat,0.016768,Meat,141.418,OUT045,2002,NaN,Tier 2,Supermarket Type1,1957.452
6952,FDN15,NaN,low fat,0.029299,Meat,140.318,OUT019,1985,Small,Tier 1,Grocery Store,139.818
8349,FDN15,NaN,low fat,0.016653,Meat,139.518,OUT027,1985,Medium,Tier 3,Supermarket Type3,2936.178


<span style="color:red">If we noticed,</span> the missing values in the `Item_Weight` may be because the data didn't enter properly,<br>
but we can impute this by the recorded item weight for each item we have

In [85]:
# Filling in the nan values with the mean of each Item_Identifier we have
train_df['Item_Weight'] = train_df.groupby('Item_Identifier')['Item_Weight'].transform(lambda x: x.fillna(x.mean()))
test_df['Item_Weight'] = test_df.groupby('Item_Identifier')['Item_Weight'].transform(lambda x: x.fillna(x.mean()))

Let's check again for any null values

In [86]:
train_df.isna().sum()

Item_Identifier                 0
Item_Weight                     4
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [87]:
test_df.isna().sum()

Item_Identifier                 0
Item_Weight                    20
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

Looking at those records that still contins a nan in the item weight

In [88]:
train_df[train_df['Item_Weight'].isna()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
927,FDN52,NaN,regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
1922,FDK57,NaN,low fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
4187,FDE52,NaN,regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
5022,FDQ60,NaN,regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


So these are only 4 items that we don't have any other records to estimate them.<br>
Let's check the mean & the median for this column to impute them

In [89]:
train_df['Item_Weight'].mean(), train_df['Item_Weight'].median()

(12.875420237117494, 12.65)

They are nearly the same, so we will use the meidan to represent the missing values

In [90]:
# Filling the nan cells with the median
train_df['Item_Weight'].fillna(train_df['Item_Weight'].median(), inplace=True)
test_df['Item_Weight'].fillna(test_df['Item_Weight'].median(), inplace=True)

In [91]:
# Checking the null again
train_df.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

Awesome, we're done with this column.<br>
Let's head to the `Outlet_Size` column and do as we did before, we will group the values by the Outlet_Identifier to see if there is any pattern

In [92]:
# taking the first 3 items for testing
t_outlets = train_df['Outlet_Identifier'].unique()[0:3]
train_df[train_df['Outlet_Identifier'] == t_outlets[0]]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,low fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
2,FDN15,17.500,low fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
10,FDY07,11.800,low fat,0.000000,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0266
12,FDX32,15.100,regular,0.100014,Fruits and Vegetables,145.4786,OUT049,1999,Medium,Tier 1,Supermarket Type1,1589.2646
17,FDP49,9.000,regular,0.069196,Breakfast,54.3614,OUT049,1999,Medium,Tier 1,Supermarket Type1,718.3982
...,...,...,...,...,...,...,...,...,...,...,...,...
8451,FDK21,7.905,low fat,0.010028,Snack Foods,249.6408,OUT049,1999,Medium,Tier 1,Supermarket Type1,3254.4304
8467,FDV31,9.800,low fat,0.000000,Fruits and Vegetables,175.2370,OUT049,1999,Medium,Tier 1,Supermarket Type1,3881.6140
8470,FDW27,5.860,regular,0.151088,Meat,155.1314,OUT049,1999,Medium,Tier 1,Supermarket Type1,1551.3140
8475,NCS17,18.600,low fat,0.080627,Health and Hygiene,92.5436,OUT049,1999,Medium,Tier 1,Supermarket Type1,378.1744


In [93]:
# The 2nd item
train_df[train_df['Outlet_Identifier'] == t_outlets[1]]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1,DRC01,5.920,regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
5,FDP36,10.395,regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
16,NCB42,11.800,low fat,0.008596,Health and Hygiene,115.3492,OUT018,2009,Medium,Tier 3,Supermarket Type2,1621.8888
31,NCS17,18.600,low fat,0.080829,Health and Hygiene,96.4436,OUT018,2009,Medium,Tier 3,Supermarket Type2,2741.7644
32,FDP33,18.700,low fat,0.000000,Snack Foods,256.6672,OUT018,2009,Medium,Tier 3,Supermarket Type2,3068.0064
...,...,...,...,...,...,...,...,...,...,...,...,...
8506,DRF37,17.250,low fat,0.084676,Soft Drinks,263.1910,OUT018,2009,Medium,Tier 3,Supermarket Type2,3944.8650
8511,FDF05,17.500,low fat,0.026980,Frozen Foods,262.5910,OUT018,2009,Medium,Tier 3,Supermarket Type2,4207.8560
8515,FDH24,20.700,low fat,0.021518,Baking Goods,157.5288,OUT018,2009,Medium,Tier 3,Supermarket Type2,1571.2880
8516,NCJ19,18.600,low fat,0.118661,Others,58.7588,OUT018,2009,Medium,Tier 3,Supermarket Type2,858.8820


In [94]:
# The 3nd item
train_df[train_df['Outlet_Identifier'] == t_outlets[2]]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
3,FDX07,19.200,regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
28,FDE51,5.925,regular,0.161467,Dairy,45.5086,OUT010,1998,NaN,Tier 3,Grocery Store,178.4344
30,FDV38,19.250,low fat,0.170349,Dairy,55.7956,OUT010,1998,NaN,Tier 3,Grocery Store,163.7868
45,FDM39,6.420,low fat,0.089499,Dairy,178.1002,OUT010,1998,NaN,Tier 3,Grocery Store,358.2004
65,FDC46,17.700,low fat,0.195068,Snack Foods,185.4266,OUT010,1998,NaN,Tier 3,Grocery Store,184.4266
...,...,...,...,...,...,...,...,...,...,...,...,...
8400,NCE07,8.180,low fat,0.021977,Household,140.6154,OUT010,1998,NaN,Tier 3,Grocery Store,425.4462
8432,FDI16,14.000,regular,0.227261,Frozen Foods,54.3640,OUT010,1998,NaN,Tier 3,Grocery Store,159.7920
8473,DRI47,14.700,low fat,0.035016,Hard Drinks,144.3128,OUT010,1998,NaN,Tier 3,Grocery Store,431.4384
8486,FDR20,20.000,regular,0.000000,Fruits and Vegetables,46.4744,OUT010,1998,NaN,Tier 3,Grocery Store,45.2744


We notice that there are some Outlets that we don't have any information about its size at all.<br>
So we consider grouping with the `Outlet_Type` and `Outlet_Location_Type` to see if there is any pattern

In [95]:
# Showing the count of non-nan values for the column Outlet_Size groubed by Outlet_Type
train_df[['Outlet_Type', 'Outlet_Size']].value_counts()

Outlet_Type        Outlet_Size
Supermarket Type1  Small          1860
Supermarket Type3  Medium          935
Supermarket Type1  High            932
                   Medium          930
Supermarket Type2  Medium          928
Grocery Store      Small           528
dtype: int64

In [96]:
# Showing the count of non-nan values for the column Outlet_Size groubed by Outlet_Location_Type
train_df[['Outlet_Location_Type', 'Outlet_Size']].value_counts()

Outlet_Location_Type  Outlet_Size
Tier 3                Medium         1863
Tier 1                Small          1458
Tier 3                High            932
Tier 1                Medium          930
Tier 2                Small           930
dtype: int64

Since there isn't any sppecific pattern, we will impute the missing values based on the **mode** value and<br>
adding a <span style="color:blue">Missing Value Indicator</span> column

In [97]:
# Adding a Missing Value Indicator column for the Outlet_Size
train_df['Outlet_Size_Nan'] = np.where(train_df['Outlet_Size'].isna(), 1, 0)
test_df['Outlet_Size_Nan'] = np.where(test_df['Outlet_Size'].isna(), 1, 0)

In [98]:
# Rearranging the columns for better understanding
column_list = list(train_df.columns)
column_list.pop(column_list.index('Outlet_Size_Nan'))
column_list.insert(column_list.index('Outlet_Size')+1, 'Outlet_Size_Nan')
train_df = train_df[column_list]
train_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Size_Nan,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,low fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,0,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,0,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,low fat,0.016760,Meat,141.6180,OUT049,1999,Medium,0,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,1,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,low fat,0.000000,Household,53.8614,OUT013,1987,High,0,Tier 3,Supermarket Type1,994.7052


Perfect, now let's fill in the missing values by the mode

In [99]:
# Calculating the mode value and fill the null values with it
train_df['Outlet_Size'].fillna(train_df['Outlet_Size'].mode()[0], inplace=True)
test_df['Outlet_Size'].fillna(test_df['Outlet_Size'].mode()[0], inplace=True)
train_df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Size_Nan,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,low fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,0,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,0,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,low fat,0.016760,Meat,141.6180,OUT049,1999,Medium,0,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,1,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,low fat,0.000000,Household,53.8614,OUT013,1987,High,0,Tier 3,Supermarket Type1,994.7052


In [100]:
train_df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Size_Nan              0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

Great, we're done with the missing values.<br>
Now let's check if there any duplicates

<a href="#top">Back to top ⬆</a>
### <span id="duplicates">Checking for Duplicates</span>

In [101]:
train_df.duplicated().sum()

0

In [102]:
test_df.duplicated().sum()

0

Awesome there is no duplicates.<br>
With that we're done with data cleaning.

<hr width="75%">

<a href="#top">Back to top ⬆</a>
## <span id="data_preprocessing">Data Preprocessing</span>

In this section, we prepare the date to be ready to train an ML model on it<br>
As we saw in the <a href="#data_exploration">Data Exploration</a> process, there are some tasks that we need to handle, in order to get an efficient ML model.<br>
The process we'll consider for our data are: <a href="#encoding">Data Encoding</a> ➡ <a href="#data_normalization">Data Normalization</a>

Back to <a href="#data_preprocessing">Data Preprocessing</a>
### <span id="encoding">Data Encoding</span>

We will start by encoding the column `Item_Fat_Content`<br>
We will use the **One-Hot encoding** method

In [103]:
# Creating a dummy variable for the column 'Item_Fat_Content'
## This If statment is just to not raise an error if cell is executed befor and the column is already dropped
 
if 'Item_Fat_Content' in train_df.columns:
    train_df['Regular'] = pd.get_dummies(train_df['Item_Fat_Content'], drop_first=True)
    train_df.drop(columns=['Item_Fat_Content'], inplace=True)
else: """"""
train_df.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Size_Nan,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Regular
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,Medium,0,Tier 1,Supermarket Type1,3735.1380,0
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,0,Tier 3,Supermarket Type2,443.4228,1
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,Medium,0,Tier 1,Supermarket Type1,2097.2700,0
3,FDX07,19.20,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Medium,1,Tier 3,Grocery Store,732.3800,1
4,NCD19,8.93,0.000000,Household,53.8614,OUT013,1987,High,0,Tier 3,Supermarket Type1,994.7052,0


In [104]:
if 'Item_Fat_Content' in test_df.columns:
    test_df['Regular'] = pd.get_dummies(test_df['Item_Fat_Content'], drop_first=True)
    test_df.drop(columns=['Item_Fat_Content'], inplace=True)
else: """"""

Done. we will do the same for the columns `Item_Type`, `Outlet_Identifier`, `Outlet_Size`, `Outlet_Location_Type` and `Outlet_Type`

In [105]:
# Encoding the for the specified columns
cols = ['Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

for c in cols:
    values = list(train_df[c].unique())[:-1]

    if c in train_df.columns:
        train_df[values] = pd.get_dummies(train_df[c], drop_first=True)
        train_df.drop(columns=[c], inplace=True)
    else: """"""

train_df.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size_Nan,Item_Outlet_Sales,Regular,Dairy,...,Breads,Starchy Foods,Others,Medium,High,Tier 1,Tier 3,Supermarket Type1,Supermarket Type2,Grocery Store
0,FDA15,9.30,0.016047,249.8092,OUT049,1999,0,3735.1380,0,0,...,0,0,0,1,0,0,0,1,0,0
1,DRC01,5.92,0.019278,48.2692,OUT018,2009,0,443.4228,1,0,...,0,1,0,1,0,0,1,0,1,0
2,FDN15,17.50,0.016760,141.6180,OUT049,1999,0,2097.2700,0,0,...,0,0,0,1,0,0,0,1,0,0
3,FDX07,19.20,0.000000,182.0950,OUT010,1998,1,732.3800,1,0,...,0,0,0,1,0,0,1,0,0,0
4,NCD19,8.93,0.000000,53.8614,OUT013,1987,0,994.7052,0,0,...,0,0,0,0,0,0,1,1,0,0


In [106]:
cols = ['Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']

for c in cols:
    values = list(test_df[c].unique())[:-1]

    if c in test_df.columns:
        test_df[values] = pd.get_dummies(test_df[c], drop_first=True)
        test_df.drop(columns=[c], inplace=True)
    else: """"""

Great. Since all the columns have been encoded we won't need the columns `Item_Identifier` and `Outlet_Establishment_Year`, so we'll drop it

In [107]:
train_df.drop(columns=['Item_Identifier', 'Outlet_Establishment_Year', 'Outlet_Identifier'], inplace=True)
test_df.drop(columns=['Item_Identifier', 'Outlet_Establishment_Year', 'Outlet_Identifier'], inplace=True)
train_df.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Size_Nan,Item_Outlet_Sales,Regular,Dairy,Soft Drinks,Meat,Fruits and Vegetables,...,Breads,Starchy Foods,Others,Medium,High,Tier 1,Tier 3,Supermarket Type1,Supermarket Type2,Grocery Store
0,9.30,0.016047,249.8092,0,3735.1380,0,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0
1,5.92,0.019278,48.2692,0,443.4228,1,0,0,0,0,...,0,1,0,1,0,0,1,0,1,0
2,17.50,0.016760,141.6180,0,2097.2700,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
3,19.20,0.000000,182.0950,1,732.3800,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,8.93,0.000000,53.8614,0,994.7052,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


**Encoding is Complete ✅**
<hr width="50%">

Back to <a href="#data_preprocessing">Data Preprocessing</a>
### <span id="data_normalization">Data Normalization</span>

Since the values of the data have a huge variations, so we consider normalizing the data before trainging the ML model on it

First Let's seperate the **sales** column as y_df and the rest as x_df

In [108]:
y_df = train_df['Item_Outlet_Sales']
x_df = train_df.drop(columns=['Item_Outlet_Sales'])
x_df.shape

(8523, 27)

In [115]:

def normalize(df):
    from sklearn.preprocessing import MaxAbsScaler
    scaler = MaxAbsScaler()
    scaler.fit(df)
    scaled = scaler.transform(df)
    scaled_df = pd.DataFrame(scaled, columns=df.columns)
    return scaled_df

X = normalize(x_df)
g_test = normalize(test_df)

In [116]:
X.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Size_Nan,Regular,Dairy,Soft Drinks,Meat,Fruits and Vegetables,Household,...,Breads,Starchy Foods,Others,Medium,High,Tier 1,Tier 3,Supermarket Type1,Supermarket Type2,Grocery Store
0,0.435597,0.048866,0.936006,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.277283,0.058705,0.180859,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,0.819672,0.051037,0.530626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.899297,0.000000,0.682289,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.418267,0.000000,0.201812,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


Pefect. Now we're ready to train our ML model<hr width="75%">


## <span id="ml">Model Training</span>

First let's split our data to training set and testing set

In [117]:
# Spliting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=0)

Now let's train our model

In [118]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

# Train a linear regression model on the training set
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model on the testing set
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print("R2 Score:", r2)
print("Mean Squared Error:", mse)

R2 Score: 0.5622107464086923
Mean Squared Error: 1281347.6152271833


The score is so low. Let's consider a **Polynomial regression** next

In [119]:
from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
poly = PolynomialFeatures(degree=2)
train_poly = poly.fit_transform(X_train)
test_poly = poly.transform(X_test)

# Train a polynomial regression model on the training set
model = LinearRegression()
model.fit(train_poly, y_train)

# Evaluate the model on the testing set
predictions = model.predict(test_poly)
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print("R2 Score:", r2)
print("Mean Squared Error:", mse)

R2 Score: 0.5970389939932671
Mean Squared Error: 1179410.2295582835


Take a look at the **Decision Tree** Algorithm and its score

In [120]:
from sklearn.tree import DecisionTreeRegressor

# Train a decision tree regression model on the training set
model = DecisionTreeRegressor(random_state=0)
model.fit(X_train, y_train)

# Evaluate the model on the testing set
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)

print("R2 Score:", r2)
print("Mean Squared Error:", mse)

R2 Score: 0.2229937158325277
Mean Squared Error: 2274188.18773466


We will pick the model with the highest accuacy wich is the **Polynomial regression**

In [122]:
# sales_predictions = model.predict(g_test)
# sales_predictions